# mapping-challenge-mask_rcnn-training
![CrowdAI-Logo](https://github.com/crowdAI/crowdai/raw/master/app/assets/images/misc/crowdai-logo-smile.svg?sanitize=true)

This notebook contains the baseline code for the training a vanilla [Mask RCNN](https://arxiv.org/abs/1703.06870) model for the [crowdAI Mapping Challenge](https://www.crowdai.org/challenges/mapping-challenge).

This code is adapted from the [Mask RCNN]() tensorflow implementation available here : [https://github.com/matterport/Mask_RCNN](https://github.com/matterport/Mask_RCNN).

First we begin by importing all the necessary dependencies : 

In [1]:
import os
import sys
import time
import numpy as np

# Download and install the Python COCO tools from https://github.com/waleedka/coco
# That's a fork from the original https://github.com/pdollar/coco with a bug
# fix for Python 3.
# I submitted a pull request https://github.com/cocodataset/cocoapi/pull/50
# If the PR is merged then use the original repo.
# Note: Edit PythonAPI/Makefile and replace "python" with "python3".
#  
# A quick one liner to install the library 
# !pip install git+https://github.com/waleedka/coco.git#subdirectory=PythonAPI

# from pycocotools.coco import COCO
# from pycocotools.cocoeval import COCOeval
# from pycocotools import mask as maskUtils

# from evaluate import build_coco_results, evaluate_coco

import zipfile
import urllib.request
import shutil


## Dataset location 
Now we have to download all the files in the datasets section and untar them to have the following structure :
```
├── data
|   ├── pretrained_weights.h5 (already included in this repository)
│   ├── test
│   │   └── images/
│   │   └── annotation.json
│   ├── train
│   │   └── images/
│   │   └── annotation.json
│   └── val
│       └── images/
│       └── annotation.json
```
Note that the `pretrained_weights.h5` (available at [https://www.crowdai.org/challenges/mapping-challenge/dataset_files](https://www.crowdai.org/challenges/mapping-challenge/dataset_files)) are the weights used for the baseline submission, and are obtained by running the learning schedule mentioned later in the experiment. In the said experiment, the initial weights used can be found [here](https://github.com/matterport/Mask_RCNN/releases/download/v2.1/mask_rcnn_balloon.h5). 

In [2]:
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils


PRETRAINED_MODEL_PATH = os.path.join(ROOT_DIR,"data/" "pretrained_weights.h5")
LOGS_DIRECTORY = os.path.join(ROOT_DIR, "logs")

Using TensorFlow backend.


## Experiment Configuration

In [3]:
from dataset import SpaceNetChallengeConfig
config = SpaceNetChallengeConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     6
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 6
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  320
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  320
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              crop
IMAGE_SHAPE                    [320 320   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0, 'rpn_bbox_loss': 1.0, 'rpn_class_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE               

## Instantiate Model

In [4]:
model = modellib.MaskRCNN(mode="training", config=config, model_dir=LOGS_DIRECTORY)
# Load pretrained weights
model_path = PRETRAINED_MODEL_PATH
model.load_weights(model_path, by_name=True)

## Load Training and Validation Dataset

In [5]:
from dataset import SpaceNetChallengeDataset

# Load training dataset
dataset_train = SpaceNetChallengeDataset()
dataset_train.load_dataset(dataset_dir="../../data", subset="train")
dataset_train.prepare()

# Load validation dataset
dataset_val = SpaceNetChallengeDataset()
val_coco = dataset_val.load_dataset(dataset_dir="../../data", subset="val")
dataset_val.prepare()

Annotation Path  ../../data/processedBuildingLabels/vectordata/summarydata/AOI_1_RIO_polygons_solution_3band.csv
Image Dir  ../../data/processedBuildingLabels/3band
RGB mean: [81.0541178  86.41590797 64.59318455]
Building Counts:
count    3879.000000
mean       59.181232
std        60.581133
min         1.000000
25%         7.000000
50%        35.000000
75%       103.000000
max       305.000000
dtype: float64
Building Widths (m):
count    229564.000000
mean         12.621780
std           7.121238
min           0.000345
25%           8.642687
50%          11.799508
75%          15.422596
max         219.500003
dtype: float64
Building Heights (m):
count    229564.000000
mean         11.937064
std           6.730427
min           0.000764
25%           8.165370
50%          11.195470
75%          14.680316
max         203.000005
dtype: float64
Annotation Path  ../../data/processedBuildingLabels/vectordata/summarydata/AOI_1_RIO_polygons_solution_3band.csv
Image Dir  ../../data/processedBu

## Train

In [ ]:
# *** This training schedule is an example. Update to your needs ***
from imgaug import augmenters as iaa
from imgaug import parameters as iap

# Inspired by SIMDRWN/YOLT: https://github.com/CosmiQ/simrdwn/blob/master/core/yolt_data_prep_funcs.py#L1003-L1182
augmentation = iaa.Sequential([
    iaa.WithColorspace(to_colorspace="HSV", from_colorspace="RGB", children=[
        iaa.WithChannels([0,1], iaa.Multiply((0.5, 1.5))),
        iaa.WithChannels(2, iaa.Multiply((0.7, 1.3)))
    ]),
    iaa.OneOf([
        iaa.Flipud(1),
        iaa.Fliplr(1),
        iaa.Affine(rotate=iap.Uniform(0, 90)),
        iaa.Affine(rotate=90),
        iaa.Affine(rotate=iap.Uniform(90, 180)),
        iaa.Affine(rotate=180),
        iaa.Affine(rotate=iap.Uniform(180, 270)),
        iaa.Affine(rotate=270),
        iaa.Affine(rotate=iap.Uniform(270, 360)),
        iaa.Affine(rotate=360),
    ])
])

# Training - Stage 1
print("Training network heads")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=10,
#             epochs=40,
            layers='heads')
#             layers='heads',
#             augmentation=augmentation)

# Training - Stage 2
# Finetune layers from ResNet stage 4 and up
print("Fine tune Resnet stage 4 and up")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=10,
#             epochs=120,
            layers='4+')
#             layers='4+',
#             augmentation=augmentation)

# Training - Stage 3
# Fine tune all layers
print("Fine tune all layers")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE / 10,
            epochs=20,
#             epochs=160,
            layers='all')
#             layers='all',
#             augmentation=augmentation)

Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /home/jupyter/crowdai-mapping-challenge-mask-rcnn/logs/spacenet-rio-buildings20190125T1618/mask_rcnn_spacenet-rio-buildings_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_cla

/home/jupyter/.local/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/10
15/90 [====>.........................] - ETA: 1:31:59 - loss: 5.9129 - rpn_class_loss: 1.3425 - rpn_bbox_loss: 2.0740 - mrcnn_class_loss: 0.7662 - mrcnn_bbox_loss: 0.7563 - mrcnn_mask_loss: 0.9740

Now you can monitor the training by running : 
```
tensorboard --logdir=logs/[path-to-your-experiment-logdir]
```
and if everything works great, you should see something like : 
![loss-plot](../../images/loss-plot.png)

# Author
Sharada Mohanty [sharada.mohanty@epfl.ch](sharada.mohanty@epfl.ch)